In [11]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
import logging
import sys
from collections.abc import Iterator
from sqlalchemy import make_url, create_engine, MetaData
from llama_index.core import ServiceContext, SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex
#from llama_index.vector_stores import PGVectorStore
import textwrap
import openai
from tqdm import tqdm
# customize textnode - purpose is to add id to each node
#from llama_index.schema import TextNode
# customize stages of querying  https://docs.llamaindex.ai/en/latest/understanding/querying/querying.html
from llama_index.core import get_response_synthesizer
from llama_index.core.indices.vector_store.retrievers.retriever import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
import pandas as pd

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
pd.set_option('max_colwidth', 2000)

In [3]:
api_key = "bac09550d21e4536b3aef7006af9d1bb"
azure_endpoint = "https://azure-open-ai-common-infra.openai.azure.com/"
api_version = "2023-07-01-preview"

# create llm and embedding model apis

llm = AzureOpenAI(
    model="gpt-4",
    deployment_name="ailab-llm",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="ada",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [4]:
# create service with llm and embedding model
from llama_index.core import set_global_service_context

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)
set_global_service_context(service_context)

C:\Users\leeto\AppData\Local\Temp\ipykernel_21916\461982355.py:4: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [7]:
# we need to write metadata for each document. In this example we do it looping a csv file
# customize document - https://docs.llamaindex.ai/en/latest/module_guides/loading/documents_and_nodes/usage_documents.html
import pandas as pd
from llama_index.core.schema import Document
df = pd.read_csv('./data/document_test/document_np_embedding_cluster.csv', encoding='utf-8-sig')

In [6]:
df.columns

Index(['Unnamed: 0', 'id', 'title', 'subtitle', 'content', 'embedding',
       'score', 'cluster'],
      dtype='object')

In [70]:
docs = []
# create list of document llama_index.schema items
from tqdm import tqdm
for i in tqdm(range(len(df))):
    curr = df.iloc[i]
    # extra_info takes meta data such as id, title, etc
    node = Document(text=curr['content'],  metadata={'id_':curr['id'],'title':curr['title'], 'subtitle':curr['subtitle']})
    docs.append(node)


100%|████████████████████████████████████████████████████████████████████████| 103836/103836 [00:55<00:00, 1860.40it/s]


In [71]:
# debugging runtime THIS TAKES 4 HOURS
import time
start = time.time()

# create index for given documents
service_context = ServiceContext.from_defaults(chunk_size=512) # control batch size 
index = VectorStoreIndex.from_documents(
    docs[:1000], service_context=service_context, show_progress=True, vector_store_query_mode="hybrid"
)
print('It took', time.time()-start, 'seconds.')

C:\Users\leeto\AppData\Local\Temp\ipykernel_21916\3407459272.py:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(chunk_size=512) # control batch size


Parsing nodes:   0%|          | 0/1000 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1003 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://azure-open

In [10]:
print('hello')

hello


In [11]:
index.storage_context.persist(persist_dir="./index_hybrid")

In [5]:
from llama_index.core import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./index_hybrid")

# load index
index = load_index_from_storage(storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.


In [74]:
df[df['title'].str.contains("Audit of the Project Management of the Food Safety Action Plan - Canadian Food Inspection Agency")]['content']


236                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [72]:
df[:1000]['title'].value_counts()

title
Convention collective entre l'Agence canadienne d'inspection des aliments et l'Institut professionnel de la fonction publique du Canada concernant l'unité de négociation du groupe Informatique (IN) - Agence canadienne d'inspection des aliments    24
D-08-04: Plant Protection Import Requirements for Plants and Plant Parts for Planting - Canadian Food Inspection Agency                                                                                                                                15
Documents proposés à l'incorporation par renvoi – Tableau canadien des ingrédients des aliments du bétail (TCIAB) - Agence canadienne d'inspection des aliments                                                                                        12
Règlement sur la santé des animaux partie XII : modification au règlement sur le transport des animauxDocument d'orientation à l'intention des parties réglementées - Agence canadienne d'inspection des aliments                                   

In [73]:
from llama_index.core.vector_stores import MetadataFilters
from llama_index.core.vector_stores import ExactMatchFilter


filters = MetadataFilters(filters=[
    ExactMatchFilter(
        key="title", 
        value='Audit of the Project Management of the Food Safety Action Plan - Canadian Food Inspection Agency'
    ),
])

In [77]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = index.as_query_engine(
    similarity_top_k=10,
    filters= filters
)

response = query_engine.query(
    "Is the following statement true? CFIA's objectives of the FSAP are to: Enhance capacity to identify risks, determine where they come from, and how to reduce and mitigate them; Provide inspectors with additional tools to prioritize their activities, verify compliance, and take enforcement actions", 
)

INFO:httpx:HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ailab-llm/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ailab-llm/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ailab-llm/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


In [78]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='98396b6b-1162-4458-98f5-e086c0472169', embedding=None, metadata={'id_': 'c7828675-ee69-40f5-be05-08810f1e2608', 'title': 'Audit of the Project Management of the Food Safety Action Plan - Canadian Food Inspection Agency', 'subtitle': 'This page has been archived'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0f5d2e2b-565f-4482-8c65-7191dc0c38af', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'id_': 'c7828675-ee69-40f5-be05-08810f1e2608', 'title': 'Audit of the Project Management of the Food Safety Action Plan - Canadian Food Inspection Agency', 'subtitle': 'This page has been archived'}, hash='212ff06e2c831f51ea9358e3fc8618abc0758244f4da9d365b07bd90dcc20b7b')}, text='This page has been archived Information identified as archived is provided for reference, research or record-keeping purposes. It is not subject to the Government of Canada Web Standards and has no

In [25]:
df[df['id'] == '08bc5d69-6638-42bd-8fe7-c988d962202b']

,Unnamed: 0,id,title,subtitle,content,embedding,score,cluster
103821,103821,08bc5d69-6638-42bd-8fe7-c988d962202b,Canadian Standards of Identity: Volume 8 – Icewine - Canadian Food Inspection Agency,Canadian Standards of Identity: Volume 8 – Icewine,"Canadian Standards of Identity: Volume 8 – Icewine Icewine 1. Icewine, ice wine or ice-wine is wine that is made exclusively from grapes naturally frozen on the vine and that, if made in Canada, has been determined by an entity acting under the authority of the law of the province in which it was made to have been made exclusively from such grapes.","[0.0135606555,-0.036940645,0.00447646,-0.05159088,0.011322426,0.0008975894,-0.023616282,0.0026632315,-0.0065768524,-0.038489684,0.01358691,0.017643293,-0.02743637,-0.011263352,-0.0036559948,0.031059546,0.018102754,-0.008926665,0.017590784,-0.019481136,-0.026517449,-0.0072200974,0.012838646,-0.012753317,-0.0045519434,0.006107546,0.009464892,-0.016684989,0.011900033,-0.010042499,0.012733626,-0.0053855362,0.001168343,0.0049063843,-0.005940171,-0.018417813,-0.0091826515,-0.01296992,0.0009328694,0.01776144,0.034840252,0.005605421,-0.00269605,0.01647495,0.00011455751,0.036152996,-0.022316664,0.02872286,0.00687222,0.011283043,0.03489276,0.00905794,-0.02095141,0.012779572,-0.00419422,-0.011821268,-0.022986164,-0.0031817656,0.007213534,-0.014571469,-0.012024744,0.00639635,-0.02990433,0.024023233,-0.0275939,-0.006754073,-0.017328234,0.012884592,0.013705058,-0.009635548,0.030718233,0.023668792,0.024272654,0.009103887,0.0021479789,0.0022890992,-0.018339047,-0.026963782,0.0017853332,-0.010226283,0.0123069845,-0.023865703,-0.015057185,0.017183831,0.0062913303,0.03016688,-0.0037281958,0.014702744,0.006248666,0.0004165914,0.003367191,0.020885773,0.0047160364,0.0059073525,-0.03344874,0.0054314826,0.033895075,0.013967606,0.0060747275,-0.019848704,0.009300798,-0.0005829408,0.007298862,-0.016763754,-0.022933654,0.012412004,0.006251948,-0.00046110168,0.014151391,0.0008943075,-0.01492591,0.036573075,0.041010153,-0.0476789,0.01585796,0.03814837,0.016448695,0.00010655797,0.011913161,-0.02769892,-0.004627426,-0.0049195117,0.03279237,0.010370686,0.014952165,0.017144449,-0.027515134,-0.019586155,0.0038135243,-0.035234075,0.028329037,-0.015844831,0.010239411,0.0061600558,-0.0034065733,0.025861077,-0.015451008,-0.0027830193,0.01868036,0.0033836002,0.00262549,0.014006989,-0.024692733,-0.0009706108,-0.008834774,-0.000644886,0.011900033,0.017800821,0.0010140955,-0.005372409,0.008624734,-0.024587713,0.026123624,0.0065243426,0.024285782,0.016658735,-0.0041843746,0.011027058,-0.006852529,0.000635...",0.616868,6709


In [15]:
pd.set_option('max_colwidth', 1000)
df['title'].value_counts()


title
Certifications biologiques révoquées - Agence canadienne d'inspection des aliments                                                                                                                                                                     558
Documents proposés à l'incorporation par renvoi – Tableau canadien des ingrédients des aliments du bétail (TCIAB) - Agence canadienne d'inspection des aliments                                                                                        557
Cancelled organic certifications - Canadian Food Inspection Agency                                                                                                                                                                                     554
Règlement sur la santé des animaux partie XII : modification au règlement sur le transport des animauxDocument d'orientation à l'intention des parties réglementées - Agence canadienne d'inspection des aliments                                

In [18]:
# configure response synthesizer
response_synthesizer = get_response_synthesizer()

query_engine = index.as_query_engine(
    similarity_top_k=15,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
    
)

response = query_engine.query("Service standard for new registration (no data) - Canadian Food Inspection Agency")

INFO:httpx:HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ailab-llm/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ailab-llm/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


In [ ]:
df[df['title']=="Service standard for new registration (no data) - Canadian Food Inspection Agency"]

In [24]:
print(response.source_nodes[0].metadata['id_'])

e5f6dd3b-89e5-4e1b-8cab-f7f9d2384e19


In [ ]:
df[df['id']=='e5f6dd3b-89e5-4e1b-8cab-f7f9d2384e19']

In [ ]:

from llama_index import StorageContext, load_index_from_storage
import time
start = time.time()

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./index")
# load index
index = load_index_from_storage(storage_context)
# configure retriever for debugging and retrieving metadata 
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=15,
    mode=VectorStoreQueryMode.HYBRID
)
# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

print('It took', time.time()-start, 'seconds.')

In [28]:
from llama_index.core import Settings
nodes = Settings.node_parser.get_nodes_from_documents(docs)

In [31]:
## build simplekeywordtableindex
from llama_index.core import SimpleKeywordTableIndex

start = time.time()
keyword_index = SimpleKeywordTableIndex.from_documents(docs, service_context=service_context, show_progress=True)
print('It took', time.time()-start, 'seconds.')

Parsing nodes:   0%|          | 0/103836 [00:00<?, ?it/s]

Extracting keywords from nodes:   0%|          | 0/104256 [00:00<?, ?it/s]

It took 166.25920867919922 seconds.


In [32]:
# import QueryBundle
from llama_index.core import QueryBundle

# import NodeWithScore
from llama_index.core.schema import NodeWithScore

# Retrievers
from llama_index.core.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
    KeywordTableSimpleRetriever,
)

from typing import List

In [33]:
class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both semantic search and hybrid search."""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        keyword_retriever: KeywordTableSimpleRetriever,
        mode: str = "AND",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._keyword_retriever = keyword_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        keyword_nodes = self._keyword_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        keyword_ids = {n.node.node_id for n in keyword_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in keyword_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(keyword_ids)
        else:
            retrieve_ids = vector_ids.union(keyword_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [34]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

# define custom retriever
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=15)
keyword_retriever = KeywordTableSimpleRetriever(index=keyword_index)
custom_retriever = CustomRetriever(vector_retriever, keyword_retriever)

# define response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
custom_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

# vector query engine
vector_query_engine = RetrieverQueryEngine(
    retriever=vector_retriever,
    response_synthesizer=response_synthesizer,
)
# keyword query engine
keyword_query_engine = RetrieverQueryEngine(
    retriever=keyword_retriever,
    response_synthesizer=response_synthesizer,
)

In [47]:
response = custom_query_engine.query(
    "Coolwater Fish Ltd. Fined $7,500 for contravening the Fish Inspection Act and Food and Drugs Act - Canadian Food Inspection Agency"
)
print(response)

INFO:httpx:HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.keyword_table.retrievers:> Starting query: Coolwater Fish Ltd. Fined $7,500 for contravening the Fish Inspection Act and Food and Drugs Act - Canadian Food Inspection Agency
> Starting query: Coolwater Fish Ltd. Fined $7,500 for contravening the Fish Inspection Act and Food and Drugs Act - Canadian Food Inspection Agency
INFO:llama_index.core.indices.keyword_table.retrievers:query keywords: ['ltd', 'act', 'inspection', 'food', 'contravening', '7', 'fish', '500', 'fined', 'coolwater']
query keywords: ['ltd', 'act', 'inspection', 'food', 'contravening', '7', 'fish', '500', 'fined', 'coolwater']
INFO:llama_index.core.indices.keyword_table.retri

In [48]:
print(response.source_nodes[0].metadata)

IndexError: list index out of range

In [42]:
response = vector_query_engine.query(
    "Service standard for new registration (no data) - Canadian Food Inspection Agency"
)
print(response)

INFO:httpx:HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ailab-llm/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ailab-llm/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
The service standard for new registration (no data) is set to be completed in 465 days. The performance target for achieving this standard is set at 90%. In the annual performance of 2018-19, 92% of 370 submissions were processed within the service standard.


In [43]:
print(response.source_nodes[0].metadata)

{'id_': 'e5f6dd3b-89e5-4e1b-8cab-f7f9d2384e19'}


In [45]:
response = keyword_query_engine.query(
    "Service standard for new registration (no data) - Canadian Food Inspection Agency"
)
print(response)

INFO:llama_index.core.indices.keyword_table.retrievers:> Starting query: Service standard for new registration (no data) - Canadian Food Inspection Agency
> Starting query: Service standard for new registration (no data) - Canadian Food Inspection Agency
INFO:llama_index.core.indices.keyword_table.retrievers:query keywords: ['registration', 'service', 'food', 'inspection', 'agency', 'standard', 'new', 'canadian', 'data']
query keywords: ['registration', 'service', 'food', 'inspection', 'agency', 'standard', 'new', 'canadian', 'data']
INFO:llama_index.core.indices.keyword_table.retrievers:> Extracted keywords: ['registration', 'service', 'food', 'inspection', 'agency', 'standard', 'new', 'canadian', 'data']
> Extracted keywords: ['registration', 'service', 'food', 'inspection', 'agency', 'standard', 'new', 'canadian', 'data']
INFO:httpx:HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ailab-llm/chat/completions?api-version=2023-07-01-preview "HT

In [46]:
print(response.source_nodes[0].metadata)

{'id_': '63095637-94e6-43cd-9c57-e1199f98a6e8'}


In [50]:
index.

AttributeError: 'VectorStoreIndex' object has no attribute 'embedding_dict'